In [103]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
import re
pd.set_option('display.max_rows', 800)


In [14]:
req = urllib.request.Request('http://www.nyc.gov/html/nypd/html/home/precincts.shtml')

In [ ]:
r = urlopen(req).read()
soup = BeautifulSoup(r)

In [40]:
tbodys = soup.find_all("tbody")

In [53]:
# get links to all precincts
hrefs = []
for tbody in tbodys:
    trs = tbody.find_all("tr")
    for tr in trs:
        a = tr.find_all("a", href=True)
        hrefs.append(a[0]['href'].strip('.'))

In [54]:
hrefs[:5]

['/precincts/precinct_001.shtml',
 '/precincts/precinct_005.shtml',
 '/precincts/precinct_006.shtml',
 '/precincts/precinct_007.shtml',
 '/precincts/precinct_009.shtml']

In [56]:
'http://www.nyc.gov/html/nypd/html'+ hrefs[0]

'http://www.nyc.gov/html/nypd/html/precincts/precinct_001.shtml'

In [ ]:
url = 'http://www.nyc.gov/html/nypd/html'+ hrefs[0]
precinct_req = urllib.request.Request(url)
precinct_soup = BeautifulSoup(urlopen(precinct_req).read())

In [70]:
print(precinct_soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<html>
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <meta content="basictext_visualeditor" name="tp-name"/>
  <meta content=" " name="keywords"/>
  <title>
   NYPD - Precincts
  </title>
  <link href="../../includes/css/rollout.css" rel="stylesheet"/>
 </head>
 <body background="../../includes/site_images/misc/bg.gif" bgcolor="#ffffff" leftmargin="0" marginheight="0" marginwidth="0" topmargin="0">
  <!-- BEGIN_PRINT_PAGE -->
  <table border="0" cellpadding="0" cellspacing="0" height="100%" width="769">
   <!-- END_PRINT_PAGE -->
   <tr>
    <td align="left" colspan="2" valign="top" width="769">
     <!-- START of nycgov_header.inc -->
     <style type="text/css">
      .header-footer-text {
	font-family:'Helvetica Neue', Helvetica, Arial, sans-serif;
	font-weight:bold;
	font-size:12px;
	line-height: 12px;

}
     </style>
     <table border="0" cellpadding="5" cellspacing="0" style="backg

In [76]:
re.findall(r"\D( \d{5})\D", precinct_soup.prettify())[0].strip()

'10013'

In [81]:
re.findall(r"\D(\d{3})\D", hrefs[0])

['001']

In [ ]:
zipcodes = {}
# loop through precinct urls
for href in hrefs:
    # get precinct number from url
    precinct = re.findall(r"\D(\d{3})\D", href)[0]
    url = 'http://www.nyc.gov/html/nypd/html'+ href
    precinct_req = urllib.request.Request(url)
    precinct_soup = BeautifulSoup(urlopen(precinct_req).read())
    # find zipcode
    zipcodes[precinct] = re.findall(r"\D( \d{5})\D", precinct_soup.prettify())[0].strip()

In [106]:
df = pd.DataFrame.from_dict(zipcodes, orient='index').reset_index()
df.columns = ['precinct','zipcode']
df.sort_values('precinct')

,precinct,zipcode
58,001,10013
22,005,10013
23,006,10014
21,007,10002
33,009,10003
42,010,10011
54,013,10010
38,014,10001
40,017,10022
2,018,10019


In [107]:
df.to_csv('precinct2zip.csv')